In [3]:
import pandas as pd
import  matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/samir/anaconda3/envs/tensorflow/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <B9202094-7D52-318C-99CF-7034B0E9F28D> /Users/samir/anaconda3/envs/tensorflow/lib/python3.11/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libom

In [ ]:
df = pd.read_csv('/Users/samir/Documents/GitHub/Google-Zero-Day-in-the-Wild-/0day _In the Wild_ - All.csv')
df.head()

,CVE,Vendor,Product,Type,Description,Date Discovered,Date Patched,Advisory,Analysis URL,Root Cause Analysis,Reported By
0,CVE-2024-23296,Apple,iOS,Memory Corruption,Arbitrary kernel r/w in RTKit,NaN,2024-03-06,https://support.apple.com/en-gb/HT214081,NaN,NaN,NaN
1,CVE-2024-23225,Apple,iOS,Memory Corruption,Out-of-bounds memory access in Kernel,NaN,2024-03-05,https://support.apple.com/en-gb/HT214081,NaN,NaN,NaN
2,CVE-2024-23222,Apple,WebKit,Memory Corruption,Type confusion,NaN,2024-01-22,https://support.apple.com/en-us/HT214059,NaN,NaN,NaN
3,CVE-2024-0519,Google,Chrome,Memory Corruption,Out-of-bounds memory access,2024-01-11,2024-01-16,https://chromereleases.googleblog.com/2024/01/...,NaN,NaN,NaN
4,CVE-2023-7024,Google,Chrome,Memory Corruption,Heap overflow in WebRTC,2023-12-19,2023-12-20,https://chromereleases.googleblog.com/2023/12/...,NaN,NaN,Clément Lecigne and Vlad Stolyarov of Google's...


In [ ]:
missing_values = df.isnull().sum()
missing_values


CVE                      0
Vendor                   0
Product                  0
Type                     4
Description              0
Date Discovered        188
Date Patched             0
Advisory                 0
Analysis URL           139
Root Cause Analysis    245
Reported By             92
dtype: int64

In [ ]:
df_cleaned = df.dropna(subset=['Date Discovered'])
df_cleaned.isnull().sum()
df_cleaned['Date Discovered'] = pd.to_datetime(df_cleaned['Date Discovered'], errors='coerce')
df_cleaned['Date Patched'] = pd.to_datetime(df_cleaned['Date Patched'], errors='coerce')
missing_type_rows =df_cleaned[df_cleaned['Type'].isnull()]

df_cleaned['Time to Patch (Days)'] = (df_cleaned['Date Patched'] - df_cleaned['Date Discovered']).dt.days

# Filter out entries where the 'Time to Patch' is negative or null
final_data = df_cleaned[(df_cleaned['Time to Patch (Days)'] >= 0) & (df_cleaned['Time to Patch (Days)'].notnull())]

# Displaying the updated dataframe
final_data[['Vendor', 'Product', 'Type', 'Time to Patch (Days)']]


/var/folders/89/xn87wdfs76x_gc0q1x5jl_4h0000gn/T/ipykernel_35828/2905786274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Date Discovered'] = pd.to_datetime(df_cleaned['Date Discovered'], errors='coerce')
/var/folders/89/xn87wdfs76x_gc0q1x5jl_4h0000gn/T/ipykernel_35828/2905786274.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Date Patched'] = pd.to_datetime(df_cleaned['Date Patched'], errors='coerce')
/var/folders/89/xn87wdfs76x_gc0q1x5jl_4h0000gn/T/ipykernel_35828/2905786

(123, 4)

Linear Regression model to predict Patch Times(Days).

In [ ]:
features = final_data[['Vendor', 'Product', 'Type']]
target = final_data['Time to Patch (Days)']
# Encoding categorical features into numerical values
label_encoders = {}
for column in features.columns:
    label_encoders[column] = LabelEncoder()
    features[column] = label_encoders[column].fit_transform(features[column])

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
X_train.head(), y_train.head()
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)


Mean Absolute Error: 31.960535042332367
Root Mean Squared Error: 35.742370036659146


/var/folders/89/xn87wdfs76x_gc0q1x5jl_4h0000gn/T/ipykernel_35828/2876078048.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[column] = label_encoders[column].fit_transform(features[column])
/var/folders/89/xn87wdfs76x_gc0q1x5jl_4h0000gn/T/ipykernel_35828/2876078048.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[column] = label_encoders[column].fit_transform(features[column])
/var/folders/89/xn87wdfs76x_gc0q1x5jl_4h0000gn/T/ipykernel_35828/2876078048.py:7: SettingWithCopyWarning: 
A

Random Forrest Regression Model.

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)


mae_rf = mean_absolute_error(y_test, y_pred)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mean Absolute Error (Random Forest):", mae_rf)
print("Root Mean Squared Error (Random Forest):", rmse_rf)

Mean Absolute Error (Random Forest): 18.32480345500774
Root Mean Squared Error (Random Forest): 31.497981919110234


XGBoost Model.

In [ ]:
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
print("Mean Absolute Error (XGBoost):", mae_xgb)
print("Root Mean Squared Error (XGBoost):", rmse_xgb)

NameError: name 'XGBRegressor' is not defined